# 3.2 翻译课程内容

## 🚅 前言
通过国际化，你的课程能够更广泛地传播，进而影响更多学习者。本节课程介绍如何通过反思方法改善较低成本的文本生成模型 Qwen-Turbo的输出质量，从而将课程内容高性价比地翻译为多种语言。

## 🍁 课程目标
学完本课程后，你将能够：
* 了解文本生成模型 Qwen-Turbo
* 了解反思方法，并使用反思法改进 Qwen-Turbo 的输出质量
  
## 📖 课程目录

- [1. 原理介绍](#🧮-1-原理介绍)
- [2. 代码实践](#🛠️-2-代码实践)
    - [2.1. 环境准备](#21-环境准备)
    - [2.2. 设置 API 客户端](#22-设置-api-客户端)
    - [2.3. 初步翻译](#23-初步翻译)
    - [2.4. 反思翻译](#24-反思翻译)
    - [2.5. 优化翻译](#25-优化翻译)
  

## 🧮 1. 原理介绍

本次课程使用了文本生成模型 [Qwen-Turbo](https://bailian.console.aliyun.com/#/model-market/detail/qwen-turbo)。它是通义千问系列速度最快、成本很低的模型，适合简单任务。

我们可以通过[反思方法](https://arxiv.org/abs/2303.11366)改善 Qwen-Turbo 的输出质量。该方法通过“生成-评估-优化”的过程实现持续改进提高模型的输出质量：

<div align="center">
    <img src="https://gw.alicdn.com/imgextra/i1/O1CN01JkN0lj1F4otLyToYM_!!6000000000434-0-tps-1004-250.jpg" alt="流程" width="35%"/>
</div>




使用反思方法提高课程内容翻译质量的过程如下：

<div align="center">
    <img src="https://gw.alicdn.com/imgextra/i2/O1CN01NjJozB1nrXxtiKQZY_!!6000000005143-0-tps-2146-170.jpg" alt="流程" width="80%"/>
</div>


## 🛠️ 2. 代码实践

接下来，让我们运行以下代码，将上一节课生成的课程内容翻译成英式英语。

### 2.1. 环境准备

1. 安装 Python 库。

In [1]:
! pip install -r requirements.txt

2. 导入必要的模块。

In [2]:
import os
from typing import List, Union, Tuple
import openai
from dotenv import load_dotenv
import json
import dashscope
from utils import create_directory, save_file, read_text_from_file,load_config

3. 设置环境变量

为了避免手动输入配置信息并提高安全性，我们推荐使用一种新的环境变量配制方法即用 load_dotenv 自动加载环境变量，例如百炼 API Key。 

这个方法可以避免将敏感信息硬编码到代码中，也无需手动配置。 具体配置方法请参考[【MAC 环境变量设置方法】](https://developer.aliyun.com/article/1490925)和[【Windows 环境变量设置方法】](https://developer.aliyun.com/article/1612143?spm=a2c6h.14164896.0.0.1ba247c5tcCw0U&scm=20140722.S_community@@%E6%96%87%E7%AB%A0@@1612143._.ID_1612143-RL_windows%E7%8E%AF%E5%A2%83%E5%8F%98%E9%87%8F-LOC_search~UND~community~UND~item-OR_ser-PAR1_213e38b017287159180232106e1660-V_3-P0_0)

当然，出于教学实验的目的，你仍然可以考虑使用交互式的方案：```os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("请输入你的api_key:")```。

你可以随自己的喜好来调整这部分课程的代码。

In [ ]:
## for MacOS users
filePath = os.path.abspath(os.path.expanduser(os.path.expandvars("~/.zshrc")))
load_dotenv(filePath)

4. 加载配置文件。

In [14]:
project_config = load_config("config.json")

### 2.2. 设置 API 客户端
设置 OpenAI 的 API 客户端，用于后续调用阿里云百炼的 Qwen-Max 模型和 Flux-Merged 模型。

In [15]:
client = openai.OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

### 2.3. 初步翻译

首先，我们让文本生成模型 Qwen-Turbo 扮演一位翻译，生成初始的译文。

1. 定义一个 `create_initial_translation` 函数，用于使用 Qwen-Turbo 将课程内容从源语言直接翻译为目标语言。

In [16]:
def create_initial_translation(source_lang, target_lang, source_text):
    """
    将源文本从指定语言翻译成目标语言，并返回翻译结果。

    参数：
    - source_lang: str，源语言代码（如 'zh' 表示中文）。
    - target_lang: str，目标语言代码（如 'en' 表示英语）。
    - source_text: str，待翻译的源文本。

    返回：
    - str，翻译后的文本。
    """
    
    # 创建系统消息，指示助手的角色和任务
    system_message = f"您是一位专业翻译，专注于将 {source_lang} 翻译成 {target_lang}。"
    
    # 将源文本和翻译任务包含在提示中，要求只输出翻译内容
    prompt = f"""您的任务是将以下文本：{source_text}从{source_lang}翻译为{target_lang}。请仅输出翻译内容，无需附加其他信息。图片路径不需要翻译。输出格式为 Markdown。"""
    
    # 通过 API 调用翻译服务，生成翻译结果
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    
    # 如果 model_dump_json() 返回字符串需要解析为 JSON 格式
    dumped_json = json.loads(completion.model_dump_json())
    
    # 从 JSON 获取生成的翻译内容
    generated_content = dumped_json['choices'][0]['message']['content']
    print("生成初步翻译:", generated_content)
    
    # 返回翻译后的文本
    return generated_content

2. 调用 `create_initial_translation` 函数，将上节课生成的“云计算”课程脚本从中文翻译为英文。

In [2]:
# 读取配置文件中的课程脚本文件路径
course_script_with_illustrations_file_path = project_config["course_script_with_illustrations_file_path"].format(title=project_config["title"])

# 根据课程脚本文件路径读取课程内容
script = read_text_from_file(course_script_with_illustrations_file_path)

# 读取配置文件中的源语言、目标语言信息
source_lang = project_config["source_lang"]
target_lang = project_config["target_lang"]

# 调用函数进行初步翻译
initial_translation = create_initial_translation(source_lang, target_lang, script)

# 读取配置文件中的初步翻译文件路径
initial_translation_file_path = project_config["initial_translation_file_path"].format(title=project_config["title"])

# 保存初步翻译结果
save_file(initial_translation, initial_translation_file_path)

### 2.4. 反思翻译

接下来，我们让 Qwen-Turbo 模型扮演语言专家，反思初始译文并提供改进建议。

1. 定义一个 `reflect_on_translation ` 函数，用于对译文进行反思。

In [18]:
def reflect_on_translation(source_lang, target_lang, source_text, translation, country):
    system_message = (
        f"您是一位语言专家，专注于从 {source_lang} 翻译到 {target_lang}。"
        f"您将获得源文本及其翻译，您的目标是改进翻译内容。"
    )
    
    prompt = f"""
    您的任务是仔细阅读从 {source_lang} 翻译到 {target_lang} 的源文本和翻译，然后提供建设性的批评和有用的建议，以改善翻译。
    
    翻译的最终风格和语气应符合在 {country} 口语中使用的 {target_lang} 的风格。
    
    源文本和翻译如下：
    <SOURCE_TEXT>
    {source_text}
    </SOURCE_TEXT>
    <TRANSLATION>
    {translation}
    </TRANSLATION>
    
    在写建议时，请注意是否有改善翻译的方式：
    (i) 准确性（通过纠正添加、误译、遗漏或未翻译文本的错误），
    (ii) 流畅性（通过应用 {target_lang} 的语法、拼写和标点规则，确保没有不必要的重复），
    (iii) 风格（确保翻译反映源文本的风格，并考虑任何文化背景），
    (iv) 术语（确保术语使用的一致性，反映源文本的领域；并仅使用与 {target_lang} 等价的习语）。
    
    写出一系列具体、有帮助和建设性的建议，以改进翻译。
    每个建议应针对翻译的一个具体部分。
    只输出建议，不加入任何额外的解释内容。
    输出格式为 Markdown。
    """

    # 直接调用 API 获取课程脚本
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    
    # 如果 model_dump_json() 返回字符串需要解析为 JSON
    dumped_json = json.loads(completion.model_dump_json())

    generated_content = dumped_json['choices'][0]['message']['content']
    print("生成翻译反思:", generated_content)
    
    # 返回翻译反思
    return generated_content

2. 调用 `reflect_on_translation` 函数生成改进建议。

In [3]:
# 读取配置文件中的国家信息
country = project_config["country"]

# 调用函数进行反思
reflection_on_translation = reflect_on_translation(source_lang, target_lang, script, initial_translation, country)

# 读取配置文件中的翻译反思文件路径
reflection_on_translation_file_path = project_config["reflection_on_translation_file_path"].format(title=project_config["title"])

# 保存翻译反思结果
save_file(reflection_on_translation, reflection_on_translation_file_path)

### 2.5. 优化翻译

接下来，我们让 Qwen-Turbo 模型扮演译审专家，进行译后编辑。

1. 定义一个 `improve_illustration` 函数，用于优化翻译。

In [20]:
def improve_translation(source_lang, target_lang, source_text, initial_translation, reflection):
    """
    改进从源语言翻译到目标语言的文本翻译。

    此函数通过使用初始翻译和改进建议，生成更准确和流畅的翻译文本。 
    函数将通过 API 调用与翻译模型进行交互，利用系统消息、源文本、初始翻译和反思建议生成最终翻译。

    参数:
    source_lang (str): 源语言的名称或代码，例如 '中文' 或 'en'。
    target_lang (str): 目标语言的名称或代码，例如 '英文' 或 'fr'。
    source_text (str): 需要翻译的源文本内容。
    initial_translation (str): 初始翻译文本，用于比较和改进。
    reflection (str): 专家的改进建议和建设性批评，供文本编辑参考。

    返回:
    str: 改进后的翻译文本，仅包含新的翻译内容。

    注意:
    函数只输出最终翻译结果，并不附加其他内容。输出格式为 Markdown。
    """

    # 定义系统消息，描述编辑任务
    system_message = (
        f"您是一位高级编辑，专注于从 {source_lang} 翻译到 {target_lang} 的后期编辑。"
        f"您将获得源文本、初始翻译和建议，您的目标是改进翻译内容。"
    )
    
    # 构建用于请求的提示信息
    prompt = f"""
    您的任务是仔细阅读并编辑从 {source_lang} 翻译到 {target_lang} 的翻译，
    同时考虑专家建议和建设性批评的列表。
    源文本：
    <SOURCE_TEXT>
    {source_text}  # 源文本内容
    </SOURCE_TEXT>
    初始翻译：
    <INITIAL_TRANSLATION>
    {initial_translation}  # 初始翻译内容
    </INITIAL_TRANSLATION>
    改进建议：
    <SUGGESTIONS>
    {reflection}  # 提供的反思或改善建议
    </SUGGESTIONS>
    
    请在编辑翻译时考虑专家建议。编辑翻译时请确保：
    (i) 准确性（纠正添加、误翻、遗漏或未翻译文本的错误），
    (ii) 流畅性（应用 {target_lang} 的语法、拼写和标点规则，确保没有不必要的重复），
    (iii) 风格（确保翻译反映源文本的风格），
    (iv) 术语（不适合上下文或使用不一致），
    (v) 其他错误。
    只输出新的翻译，不附加其他内容。
    输出格式为Markdown。
    """
    
    # 直接调用 API 获取课程脚本
    completion = client.chat.completions.create(
        model="qwen-turbo",  # 使用的模型
        messages=[
            {"role": "system", "content": system_message},  # 系统消息
            {"role": "user", "content": prompt},  # 用户提示
        ],
    )
    
    # 解析从 API 返回的 JSON 数据
    dumped_json = json.loads(completion.model_dump_json())
    # 提取生成的内容
    generated_content = dumped_json['choices'][0]['message']['content']
    
    # 打印生成的优化后的翻译
    print("生成优化后的翻译:", generated_content)
    
    # 返回优化后的翻译内容
    return generated_content

2. 调用 `improve_illustration` 函数优化翻译。

In [4]:
# 调用函数进行反思
improved_translation = improve_translation(source_lang, target_lang, script, initial_translation, reflection_on_translation)

# 读取配置文件中的优化翻译文件路径
improved_translation_file_path = project_config["improved_translation_file_path"].format(title=project_config["title"])

# 保存优化后的翻译结果
save_file(improved_translation, improved_translation_file_path)

## ✅ 本节小结

- 在本次学习和实践中，我们了解了低成本的 Qwen-Turbo，并通过反思方法改进了 Qwen-Turbo 的输出质量。
- 除了国际化的需求外，将课程内容转换为 PPT 进行授课也是非常常见的需求。接下来，我们将学习如何使用大模型结合工具将课程内容转换为 PPT。